In [5]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import time

In [6]:
bitcoin_prices = pd.read_csv('/Users/admin/Desktop/UChicago/Real Time Intelligent System/BitCoin_hist_data.csv')
bitcoin_prices.date = pd.to_datetime(bitcoin_prices.date)
bitcoin_prices = bitcoin_prices[bitcoin_prices.date.dt.year >= 2014]

In [7]:
bitcoin_gtrends = pd.read_csv('/Users/admin/Desktop/UChicago/Real Time Intelligent System/google_trends_bitcoin_2014.csv')
bitcoin_gtrends.date = pd.to_datetime(bitcoin_gtrends.date)

In [8]:
bitcoin_tweet_volumes = pd.read_csv('/Users/admin/Desktop/UChicago/Real Time Intelligent System/tweet_volumes_bitcoin_2014.csv')
bitcoin_tweet_volumes = bitcoin_tweet_volumes[['date','num_tweets']]
bitcoin_tweet_volumes.date = pd.to_datetime(bitcoin_tweet_volumes.date, format = '%Y-%m-%d')

In [9]:
bitcoin_tweet_volumes.num_tweets = bitcoin_tweet_volumes.num_tweets.interpolate()

In [10]:
bitcoin_gtrends

,date,bitcoin_unscaled,bitcoin_monthly,isPartial,scale,bitcoin
0,2014-01-01,63,8.0,False,0.08,5.04
1,2014-01-02,68,8.0,NaN,0.08,5.44
2,2014-01-03,77,8.0,NaN,0.08,6.16
3,2014-01-04,63,8.0,NaN,0.08,5.04
4,2014-01-05,71,8.0,NaN,0.08,5.68
...,...,...,...,...,...,...
2917,2021-12-27,43,35.0,NaN,0.35,15.05
2918,2021-12-28,46,35.0,NaN,0.35,16.10
2919,2021-12-29,45,35.0,NaN,0.35,15.75
2920,2021-12-30,44,35.0,NaN,0.35,15.40


# w/ google trends & tweet volumes

In [11]:
df_tmp = bitcoin_prices.merge(bitcoin_gtrends, on = 'date')[['date','close','bitcoin']]
df = df_tmp.merge(bitcoin_tweet_volumes, on = 'date')
df = df.set_index('date')

In [12]:
df.corr()

,close,bitcoin,num_tweets
close,1.000000,0.727929,0.801102
bitcoin,0.727929,1.000000,0.785577
num_tweets,0.801102,0.785577,1.000000


In [14]:
i = 1
new_df = pd.concat([df[['bitcoin','num_tweets']].shift(-i),df['close']], axis = 1).dropna()
new_df['prev_price'] = new_df['close'].shift(1).dropna()
new_df['prev_tweet_volume'] = new_df['num_tweets'].shift(1).dropna()
new_df = new_df.dropna()
X = new_df[['bitcoin','num_tweets','prev_price','prev_tweet_volume']]
y = new_df['close']

In [45]:
#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 42)
X_train = X[X.index <= pd.to_datetime('2019-12-31')]
X_test = X[X.index > pd.to_datetime('2019-12-31')]
y_train = y[y.index <= pd.to_datetime('2019-12-31')]
y_test = y[y.index > pd.to_datetime('2019-12-31')]

In [46]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.fit_transform(X_test)

In [47]:
tree = DecisionTreeRegressor(min_samples_split = 10, min_samples_leaf=5)
tree.fit(X_train_scaled, y_train)

DecisionTreeRegressor(min_samples_leaf=5, min_samples_split=10)

In [46]:
import pickle

pickle.dump(lr, open('lr_tweet_bitcoin.pkl', 'wb'))

In [104]:
# try with cv?

for i in range(8):
    new_df = pd.concat([df[['bitcoin','num_tweets']].shift(-i),df['close']], axis = 1).dropna()
    new_df['prev_price'] = new_df['close'].shift(1)
    new_df['prev_tweet_volumne'] = new_df['num_tweets'].shift(1)
    X = new_df[['bitcoin','num_tweets']]
    y = new_df['close']
    ss = StandardScaler()
    X_scaled = ss.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3)
#     X_train = X[:2093]
#     X_test = X[2094:]
#     y_train = y[:2093]
#     y_test = y[2094:]
    tree = DecisionTreeRegressor(min_samples_split = 10, min_samples_leaf=5)
    tree.fit(X_train, y_train)
    y_train_pred = tree.predict(X_train)
    y_test_pred = tree.predict(X_test)
    print('Training R2 score shifting back predictors {} days: {}'.format(i,r2_score(y_train, y_train_pred)))
    print('Testing R2 score shifting back predictors {} days: {}'.format(i,r2_score(y_test, y_test_pred)))
    print('--- --- --- ---')

Training R2 score shifting back predictors 0 days: 0.9034524939800106
Testing R2 score shifting back predictors 0 days: 0.6965074058612004
--- --- --- ---
Training R2 score shifting back predictors 1 days: 0.8901760705406999
Testing R2 score shifting back predictors 1 days: 0.7507351304492594
--- --- --- ---
Training R2 score shifting back predictors 2 days: 0.890227556914271
Testing R2 score shifting back predictors 2 days: 0.7140334000332602
--- --- --- ---
Training R2 score shifting back predictors 3 days: 0.885018783015717
Testing R2 score shifting back predictors 3 days: 0.7253445528227108
--- --- --- ---
Training R2 score shifting back predictors 4 days: 0.8875298514306544
Testing R2 score shifting back predictors 4 days: 0.6991975699659967
--- --- --- ---
Training R2 score shifting back predictors 5 days: 0.8904964129727138
Testing R2 score shifting back predictors 5 days: 0.7165174060494975
--- --- --- ---
Training R2 score shifting back predictors 6 days: 0.8917531054765961
Te

In [147]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

i = 3
new_df = pd.concat([df[['bitcoin','num_tweets']].shift(-i),df['close']], axis = 1).dropna()
new_df['prev_price'] = new_df['close'].shift(1)
new_df['prev_tweet_volumne'] = new_df['num_tweets'].shift(1)
X = new_df[['bitcoin','num_tweets']]
y = new_df['close']
ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2)
    
modelparams = {RandomForestRegressor() : {'min_samples_split' : [5,10,20], 'min_samples_leaf' : [5,10]}
              ,DecisionTreeRegressor() : {'min_samples_split' : [5,10,20], 'min_samples_leaf' : [5,10]}
              ,LinearSVR() : {'epsilon' : [0, 0.01, 0.02, 0.03]}
              ,GradientBoostingRegressor() : {'n_estimators' : [100, 200, 300], 'learning_rate' : [0.1, 0.2, 0.5], 'min_samples_split' : [5,10,20]}
              ,LinearRegression() : {}
              }

for model, params in tqdm(modelparams.items()):
    clf = GridSearchCV(model, params, cv = 5)
    clf.fit(X_train, y_train)
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    print(clf.best_estimator_)
    print("R2:", round(clf.best_score_,2))
    print("Test RMSE:", round(mean_squared_error(y_test, y_test_pred, squared = False),2))
    print("Test MAPE:", round(mean_absolute_percentage_error(y_test, y_test_pred),2))

  0%|          | 0/5 [00:00<?, ?it/s]

RandomForestRegressor(min_samples_leaf=10, min_samples_split=20)
R2: 0.79
Test RMSE: 7067.11
Test MAPE: 0.63
DecisionTreeRegressor(min_samples_leaf=10, min_samples_split=5)
R2: 0.76
Test RMSE: 7773.23
Test MAPE: 0.64
LinearSVR(epsilon=0.01)
R2: -0.26
Test RMSE: 18238.34
Test MAPE: 0.75
GradientBoostingRegressor(min_samples_split=5)
R2: 0.79
Test RMSE: 7089.81
Test MAPE: 0.66
LinearRegression()
R2: 0.65
Test RMSE: 9060.98
Test MAPE: 3.63
